In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gresearch_crypto
import matplotlib.pyplot as plt
env = gresearch_crypto.make_env()
from datetime import datetime
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Model build and submission

The steps for data creation can be found here: 

[https://www.kaggle.com/craniket/g-research-data-creation](http://).

Here I build a basic LGBM model and submit the predictions.

In [ ]:
data_dir1 = "/kaggle/input/training/"
x_train = pd.read_csv(data_dir1+"x_train.csv")
y_train = pd.read_csv(data_dir1+"y_train.csv")
y_test = pd.read_csv(data_dir1+"y_test.csv")
weight_train = pd.read_csv(data_dir1+"weight_train.csv")
weight_test = pd.read_csv(data_dir1+"weight_test.csv")

In [ ]:
data_dir2 = "/kaggle/input/testxx/"
x_test = pd.read_csv(data_dir2+"x_test.csv")

Dropping unnecessary columns

In [ ]:
x_train = x_train.drop(["timestamp","filt"],axis = 1)
x_test = x_test.drop(["timestamp","filt"],axis = 1)
y_train = y_train.drop(["timestamp"],axis = 1)
y_test = y_test.drop(["timestamp"],axis = 1)
x_train['Mean'] = x_train[['Open', 'High', 'Low', 'Close']].mean(axis=1)    
x_train['High/Mean'] = x_train['High'] / x_train['Mean']
x_train['Low/Mean'] = x_train['Low'] / x_train['Mean']
x_train['Volume/Count'] = x_train['Volume'] / (x_train['Count'] + 1)
weight_train = weight_train.drop(["timestamp"],axis = 1).values
weight_test = weight_test.drop(["timestamp"],axis = 1).values

In [ ]:
import lightgbm as lgb

Here I define the metrics as per rules, but in practice implementing this on test data took up too much space.

In [ ]:
def wmean(x, w):
    return np.sum(x * w) / np.sum(w)

def wcov(x, y, w):
    return np.sum(w * (x - wmean(x, w)) * (y - wmean(y, w))) / np.sum(w)

def wcorr(x, y, w):
    return wcov(x, y, w) / np.sqrt(wcov(x, x, w) * wcov(y, y, w))

def eval_wcorr(preds, dataset):
    y =dataset.get_label()    
    w = dataset.get_weight()
    return 'eval_wcorr', wcorr(preds, y, w), True

params = {'n_estimators': 500,
        'objective': 'regression_l1',  'metric': 'None',
        'boosting_type': 'gbdt',
        'max_depth': -1,
        'learning_rate': 0.001,
        'subsample': 0.4,
        'subsample_freq': 4,
        'feature_fraction': 0.4,
        'lambda_l1': 1,
        'lambda_l2': 1,
        'seed': 123,
        'verbose': -1,
        }
pred = []
score=[]
evals_result = {}
models = []
test_dataset = lgb.Dataset(x_test, y_test,weight = weight_test)
for idx_t,idx_v in KFold(n_splits = 3, shuffle = True, random_state = 2).split(x_train):
    X_t,y_t,X_v,y_v = x_train.iloc[idx_t],y_train.iloc[idx_t],x_train.iloc[idx_v],y_train.iloc[idx_v]
    weight_t,weight_v = weight_train[idx_t],weight_train[idx_v]
    train_dataset = lgb.Dataset(X_t, y_t,weight = weight_t)
    val_dataset = lgb.Dataset(X_v, y_v,weight =weight_v)    
    model = lgb.train(params,
                          #early_stopping_rounds=1000,
                          verbose_eval = 100,
                          feval=eval_wcorr,
                          train_set = train_dataset, 
                          valid_sets = [val_dataset],
                          evals_result = evals_result 
                         )
    models.append(model)
    #preds2 = model.predict(x_test)
    #sc = eval_wcorr(preds2, test_dataset)
    #score = score.append(sc)
    

In [ ]:
#preds2.shape

In [ ]:
del x_train
del x_test
del train_dataset
del test_dataset
del val_dataset
del y_train
del y_test
import gc
gc.collect()

In [ ]:
iter_test = env.iter_test()# an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    test_df['change'] = test_df['Close']/test_df['Open']
    test_df['variation'] = test_df['High']/test_df['Low']
    test_df['change_diff'] = test_df['Close'] - test_df['Open']
    test_df['variation_diff'] = test_df['High'] - test_df['Low']
    def hlco_ratio(df): return (df['High'] - df['Low'])/(df['Close']-df['Open'])
    def upper_shadow(df): return df['High'] - np.maximum(df['Close'], df['Open'])
    def lower_shadow(df): return np.minimum(df['Close'], df['Open']) - df['Low']
    test_df['Upper_Shadow'] = upper_shadow(test_df)
    test_df['Lower_Shadow'] = lower_shadow(test_df)
    test_df['hlco_ration'] = hlco_ratio(test_df)
    test_df['Mean'] = test_df[['Open', 'High', 'Low', 'Close']].mean(axis=1)    
    test_df['High/Mean'] = test_df['High'] / test_df['Mean']
    test_df['Low/Mean'] = test_df['Low'] / test_df['Mean']
    test_df['Volume/Count'] = test_df['Volume'] / (test_df['Count'] + 1)

    preds2 = []
    for i in range(3):
        model = models[i]
        preds = model.predict(test_df.drop(["timestamp","row_id"],axis=1))
        preds2.append(preds)
    sample_prediction_df['Target'] = np.mean(np.column_stack(preds2),axis=1)  # make your predictions here
    env.predict(sample_prediction_df) 